# Desafío AB InBev

## [0] Librerías necesarias para el proyecto

In [1]:
# Conectamos Google Drive con Google Colab
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
# Librerías necesarias para el desarrollo del proyecto
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
import scipy.sparse
from collections import defaultdict
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize
import sys
import os
import time
import multiprocessing as mp
from tqdm import tqdm
from datetime import date

## [1] Entendimiento de los datos

Etapa inicial en un proyecto de Ciencia de Datos, nos permite explorar las características de los datos a ser modelados.

In [3]:
# Lectura de los datos (formato parquet)
data = pd.read_parquet('/gdrive/My Drive/AB_InBev/datos/dataset_test.parquet', engine='pyarrow')

In [4]:
# Creamos una lista de campos que serán analizados
vars = data.columns.to_list()
vars

['Date', 'Account_id', 'Product_id', 'Category', 'Quantity']

In [5]:
# Agrupamos la data (está expresada a nivel transaccional) a nivel cliente - producto - día
data = data.groupby(vars[:-1])[vars[-1]].sum().reset_index()

In [6]:
# Cambiamos el campo Date de objeto a fecha
data['Date'] = data['Date'].astype('datetime64[ns]')

In [7]:
# Generamos el resumen de la cantidad de clientes a nivel día
res_date = data.groupby(['Date'])['Account_id'].count().reset_index()

In [9]:
# Calculamos las fechas que presentan un comportamiento atípico a nivel de clientes
p75 = np.quantile(res_date['Account_id'],0.75)
p25 = np.quantile(res_date['Account_id'],0.25)
ric = p75 - p25
l_inf = p25 - (1.5*ric)
l_sup = p75 - (1.5*ric)

In [10]:
# Extraemos los campos Year y Month desde el campo Date
data['Year'] = data['Date'].dt.year 
data['Month'] = data['Date'].dt.month 

In [11]:
# Guardamos los datos agrupados a nivel cliente - producto - día
#data.to_csv('/gdrive/My Drive/AB_InBev/datos/data_agrupada.csv',header='True',index_label='False')

## [2] Preparación de los datos

Etapa en la que limpiamos los datos para simplificar la complejidad natural de la búsqueda de patrones por parte de los algoritmos.

In [12]:
# Calculamos los rangos intercuartílicos para la identificación de outliers a nivel de cantidad de compras

q3 = np.quantile(data['Quantity'],0.75)
q1 = np.quantile(data['Quantity'],0.25)
ric = q3 - q1
lim_sup = q3 + (1.5*ric)
lim_inf = q1 - (1.5*ric)

In [13]:
# Límites según criterio de Boxplot
lim_inf, lim_sup

(-578.5, 1201.5)

In [14]:
# Filtramos los casos que se encuentran contenidos entre los límites
data = data[(data['Quantity']>= lim_inf) & (data['Quantity']<= lim_sup)]

In [15]:
# Analizamos la cantidad de compradores por día
data_comprador = data.groupby(['Date'])['Account_id'].nunique().reset_index()


In [16]:
# Calculamos la cantidad de compradores atípicos por día
perc75 = np.quantile(data_comprador.Account_id.values,0.75)
perc25 = np.quantile(data_comprador.Account_id.values,0.25)
ric = perc75 - perc25
l_inf = perc25 - (1.5*ric)
l_sup = perc75 + (1.5*ric)
l_inf, l_sup

(610.25, 2292.25)

In [17]:
# Analizamos los días en los que no hubieron cantidad atípica de compradores (menores al límite inferior)
data_fec_out = data_comprador[(data_comprador['Account_id']<l_inf)]

In [18]:
# Excluimos a los días en los que hubieron pocos compradores
data = data[(~data['Date'].isin(data_fec_out.Date.values))]

In [19]:
# Guardamos los datos agrupados sin outliers
#data.to_csv('/gdrive/My Drive/AB_InBev/datos/data_agrupada_so.csv',header='True',index= False,index_label=False)

In [20]:
# Calculamos la cantidad de meses
n_meses = data.Month.max()
n_meses

7

In [21]:
# Separamos los datos para entrenar el modelo (data_train) y para validarlo (data_test)
data_train = data[(data['Month']<=(n_meses))]

In [22]:
# Exportamos los datos de entrenamiento y validación
#data_train.to_csv('/gdrive/My Drive/AB_InBev/datos/data_train.csv',header='True',index= False,index_label=False)

In [23]:
###################
#### Temporal
###################

#data_train = pd.read_csv('/gdrive/My Drive/AB_InBev/datos/data_train.csv')

In [24]:
# Agrupamos las compras a nivel cliente - producto - cantidad de productos comprados por día
data_train_sr = data_train.groupby(['Account_id','Product_id'])['Quantity'].mean().reset_index()
data_train_sr.head(10)

,Account_id,Product_id,Quantity
0,33214647,8222,123.923077
1,33214647,8352,92.250000
2,33214647,8358,89.000000
3,33214647,14014,248.181818
4,33214647,16578,83.750000
5,33214647,16582,311.500000
6,33214647,17468,311.500000
7,33214647,22270,85.500000
8,33217422,8230,84.428571
9,33217422,8320,89.000000


In [25]:
# Creamos una lista ordenada en base al id del cliente
clients = list(sorted(set(data_train_sr['Account_id'].values)))
data_train_sr['cliente'] = np.searchsorted(clients, data_train_sr['Account_id'], side='left')


In [27]:
# Creamos una lista ordenada en base al id del producto
products = list(sorted(set(data_train_sr['Product_id'].values)))
data_train_sr['sku'] = np.searchsorted(products, data_train_sr['Product_id'], side='left')

In [28]:
# Definimos como índices a los clientes y productos comprados
data_train_sr = data_train_sr.set_index(['cliente', 'sku'], drop=False)

In [29]:
# Exportamos los datos
#data_train_sr.to_csv('/gdrive/My Drive/AB_InBev/datos/data_train_sr.csv',header='True',index= False,index_label=False)

In [30]:
# Creamos una matriz dispersa para el proceso de aprendizaje del algoritmo
sparse_data = scipy.sparse.csc_matrix((
    data_train_sr['Quantity'].values.astype(dtype=np.float64), 
    (data_train_sr['cliente'].values, data_train_sr['sku'].values)
))

In [31]:
# Normalizamos la matriz dispersa
sparse_data = normalize(sparse_data, axis=1)

In [ ]:
# Eliminar datasets pesados
del data
del data_comprador
del data_fec_out

In [33]:
# Factorización matricial, generando tres matrices insumo para el algoritmo
U, sigma, Vt = svds(sparse_data, k = 150, maxiter=30000) 

In [34]:
# Creamos una matriz diagonal
sigma = np.diag(sigma)

In [35]:
# Creamos el data frame de productos con sus identificadores
promos = pd.DataFrame(products, columns=['id_sku'])

In [36]:
# Buscamos los índices de los productos que buscamos recomendar (homologación de índices)
start_time = time.time()

promos2 = promos[promos['id_sku'].isin(products)] #lo que esta en promos lo busca en products, si está lo retorna
promos2['idx_sku'] = np.searchsorted(products, promos2['id_sku'].values)

print("Iniciando Transformaciones: ", time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time)))

Iniciando Transformaciones:  00:00:00


In [37]:
# Filtramos los productos que buscamos recomendar en la matriz de compra de productos
promotional_products_idx = list(promos2['idx_sku'].values)
product_embeds = np.take(Vt, promotional_products_idx,axis=1)

In [38]:
# Definimos los clientes a los que aplicaremos las recomendaciones
customers = pd.DataFrame(data_train_sr.Account_id.unique(),columns=['Account_id'])
customers_2 = customers[customers['Account_id'].isin(clients)]
customers_2['idx_corp'] = np.searchsorted(clients, customers_2['Account_id'].values)
a = list(customers_2['idx_corp'].values)

In [39]:
# Filtra a todos los usuarios de la matriz U (matriz de compras clientes)
promo_mail_stats = {}
r_clients_preferences_embed = np.take(U, a, axis=0)

In [40]:
# Construye la matriz M (usuarios y productos)
r_c_product_recommendations = np.dot(np.dot(r_clients_preferences_embed, sigma), product_embeds)

In [41]:
# Validamos la dimensionalidad
r_c_product_recommendations.shape

(12534, 181)

## [3] Entrenamiento de algoritmo

Etapa en la que entrenamos al algoritmo y este se encarga de la búsqueda de patrones que permitirán predecir eventos futuros.

In [42]:
# Función que permite generar recomendaciones a nivel cliente
def mass_recomm(promoss, client_preferences, promos_id_sku, ID_CORP, n_recom=15):
    promoss['score'] = np.take(client_preferences,  np.arange(promoss.shape[0]), axis=0)
    promos_a = promoss.groupby(['id_sku'])['score'].max().reset_index() 
    promos_a_s = promos_a.sort_values('score', ascending=False).reset_index(drop=True).reset_index() 
    promos_a_s['index'] += 1 
    promos_a_s['ID_CORP'] = ID_CORP
    # return promos_a_s.head(int(sys.argv[2]))
    return promos_a_s.head(int(n_recom))
    # return promos_a_s

In [43]:
n_recom = 5

In [44]:
# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.starmap` the `mass_recomm()`
results = pool.starmap(mass_recomm, tqdm([(promos2, r_c_product_recommendations[client_idx], promotional_products_idx, clients[a_idx], n_recom) for client_idx, a_idx in zip(list(range(len(a))), a)]))

# Step 3: Don't forget to close
pool.close()

promo_mails = pd.concat(results, axis=0)
del results

100%|██████████| 12534/12534 [00:21<00:00, 573.04it/s]


In [45]:
# Descriptivo de las estadísticas del score
promo_mails['score'].describe().T

count    62670.000000
mean         0.340052
std          0.173931
min          0.000010
25%          0.250467
50%          0.316928
75%          0.412188
max          1.000000
Name: score, dtype: float64

In [46]:
# Cantidad de usuarios únicos
promo_mails.ID_CORP.nunique()

12534

## [4] Scoring

In [49]:
output = pd.DataFrame([])
output['Account_id'] = promo_mails['ID_CORP']

In [50]:
output['Product_id'] = promo_mails['id_sku']

In [51]:
output.head(10)

,Account_id,Product_id
0,33214647,17468
1,33214647,16582
2,33214647,14014
3,33214647,8222
4,33214647,8352
0,33217422,8328
1,33217422,8320
2,33217422,8356
3,33217422,8334
4,33217422,8230


In [52]:
output.to_csv('/gdrive/My Drive/AB_InBev/datos/output_sr.csv',header='True',index= False,index_label=False)